In [10]:
import logging
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s: (%(lineno)s) %(levelname)s %(message)s")

import json
import pprint

import networkx as nx

from semeval_fca import find_paths_to_object

# Tree exploration

In [9]:
J = json.load(open(
    '/mnt/store/hlt/Work/hypernym18-SemEval/concept_tree/1A_UMBC_tokenized.txt_100_sg.vec.gz_True_1000_0.2_unit_True_vocabulary_filtered_reduced.cxt.json'))

In [11]:
paths, nodes, depths = find_paths_to_object(J, 'dhow')

mind3 visszaadott lista olyan hosszú lesz, mint ahányféleképp el lehetett jutni a gyökérből egy olyan csúcsig, amiben megtalálható volt a lekérdezett szó

* a `path` ezeknek az utaknak a visszafejtését tartalmazza, 
* a `nodes` magát azt a csúcsot, ahol az i. út véget ért
* a `depths` pedig azt hogy a gyökértől milyen mélyen helyezkedik el ez a bizonyos csúcs


elvileg redundáns is, tehát a `paths` i. elemében egy olyan listát kell találj, ami egy gyökérben végződő utat kódol, és a `depths` i. értékéül épp ennek az útnak a hosszát kell lásd

In [12]:
len(paths), len(nodes), len(depths)

(22, 22, 22)

In [24]:
paths[0]

[{'Node': 0, 'attributes': [], 'own_objects': []}]

In [25]:
nodes[0][

['watercraft',
 'edgily',
 'nitro',
 'road_racing',
 'sports_car',
 'dinky-di',
 'jalopy',
 'cupholder',
 'lawnmower',
 'hardtop',
 't0',
 'sidesaddle',
 'tyre',
 'crew_cab',
 'joyride',
 'bonkers',
 'motorcycle',
 'coachbuilder',
 'cruiser',
 'military_vehicle',
 'border_collie',
 'prius',
 'clutch_pedal',
 'off-roader',
 'chinky',
 'circular_segment',
 'hardpack',
 'minivan',
 'kickflip',
 'drowsiness',
 'power_train',
 'sailboat',
 'trunk_lid',
 'aks',
 'floorboard',
 'underbody',
 'three-wheeled',
 'stunna',
 'biker',
 'upholstery',
 'soft_top',
 'car',
 'racing_circuit',
 'curricle',
 'phaeton',
 'racetrack',
 'saddle_horse',
 'tranny',
 'chevy',
 'flat_tire',
 'roll_cage',
 'econocar',
 'lawn-mower',
 'defroster',
 'truckish',
 'accelerator_pedal',
 'slappy',
 'cabriolet',
 'attack_helicopter',
 'up-armored',
 'grippy',
 'horse_trainer',
 'air_suspension',
 'oil_slick',
 'fender',
 'manumatic',
 'ignition_switch',
 'grand_tourer',
 'braking',
 'skateboard',
 'roadgoing',
 'flat-s

In [26]:
depths[0]

1

# Lattice exploration

In [2]:
dag = nx.read_gpickle('/mnt/permanent/home/makrai/project/hypernym18-SemEval/top500words.gpickle')

In [4]:
dag.number_of_nodes(), dag.number_of_edges()0

(196, 446)

In [8]:
print(dag.nodes['node175']['label'])

175|0th|n816\nn288\nn106\nn463\nn954\nn527\nn150\nn226\nn533\nn36
